<a href="https://colab.research.google.com/github/Jihaad2021/Paractical_Keras/blob/main/Natural_Language_Processing/03_Text_CLassification_Using_Pre_trained_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup 
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 

In [2]:
# download the newsgroup20 data 
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 9s 1us/step


In [4]:
import os 
import pathlib 

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories: ", len(dirnames)) 
print("Directory names: ", dirnames) 

fnames = os.listdir(data_dir/"comp.graphics")
print("Number of files in comp.graphics: ", len(fnames))
print("Some example filenames: ", fnames[:5])


Number of directories:  20
Directory names:  ['rec.motorcycles', 'misc.forsale', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'soc.religion.christian', 'sci.space', 'rec.sport.baseball', 'talk.politics.misc', 'comp.windows.x', 'sci.med', 'talk.religion.misc', 'talk.politics.guns', 'rec.autos', 'sci.crypt', 'talk.politics.mideast', 'comp.sys.mac.hardware', 'rec.sport.hockey', 'comp.os.ms-windows.misc', 'sci.electronics', 'alt.atheism']
Number of files in comp.graphics:  1000
Some example filenames:  ['37955', '38810', '39061', '39081', '39014']


In [5]:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



In [9]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [10]:
# shuffle and split the data into training and validation sets

# shuffle the data 
seed = 1337 
rng = np.random.RandomState(seed) 
rng.shuffle(samples)

rng = np.random.RandomState(seed)
rng.shuffle(labels) 

# extract a training & validation split 
validation_split = 0.2 
num_validation_samples = int(validation_split * len(samples)) 

train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]

train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]


In [12]:
# creating a vocabulary index 

from tensorflow.keras.layers import TextVectorization 

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [14]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [15]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0,:6]

array([   2, 3678, 1683,   15,    2, 7192])

In [22]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc)))) 

In [23]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3678, 1683, 15, 2, 7192]

In [16]:
# Load pre-trained word embeddings 
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-05-11 21:01:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-11 21:01:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-11 21:01:58--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [24]:
# Let's make a dict mapping words (strings) to their NumPy vector representation:

path_to_glove_file = "/content/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


Found 400000 word vectors.


In [26]:
num_tokens = len(voc)+2 
embedding_dim = 100 
hits = 0 
misses = 0

# prepare embedding matrix 
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items(): 
  embedding_vector = embeddings_index.get(word) 
  if embedding_vector is not None: 
    embedding_matrix[i] = embedding_vector 
    hits += 1 
  else:
    misses +=1 

print("Converted %d words (%d misses)" % (hits, misses))

Converted 18002 words (1998 misses)


In [35]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)


In [38]:
# Build the model 

from tensorflow.keras import layers 
int_sequence_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequence_input) 
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences) 
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x) 
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x) 
model = keras.Model(int_sequence_input, preds) 
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d_6 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [39]:
# train the model 
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy() 
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy() 

y_train = np.array(train_labels) 
y_val = np.array(val_labels)

In [40]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 23s 176ms/step - loss: 2.6865 - acc: 0.1319 - val_loss: 2.1035 - val_acc: 0.2848
Epoch 2/20
125/125 [==============================] - 22s 175ms/step - loss: 1.9979 - acc: 0.3046 - val_loss: 1.5798 - val_acc: 0.4484
Epoch 3/20
125/125 [==============================] - 23s 188ms/step - loss: 1.5736 - acc: 0.4523 - val_loss: 1.2896 - val_acc: 0.5584
Epoch 4/20
125/125 [==============================] - 21s 167ms/step - loss: 1.2923 - acc: 0.5557 - val_loss: 1.1510 - val_acc: 0.6002
Epoch 5/20
125/125 [==============================] - 23s 180ms/step - loss: 1.1192 - acc: 0.6176 - val_loss: 1.3401 - val_acc: 0.5609
Epoch 6/20
125/125 [==============================] - 22s 180ms/step - loss: 0.9867 - acc: 0.6641 - val_loss: 1.2222 - val_acc: 0.6042
Epoch 7/20
125/125 [==============================] - 21s 172ms/step - loss: 0.8726 - acc: 0.6991 - val_loss: 1.0552 - val_acc: 0.6434
Epoch 8/20
125/125 [==============================] - 2

In [41]:
# export an end-to=end model 

string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input) 
preds = model(x) 
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)
class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 156ms/step


'comp.graphics'